# Required Libraries

In [ ]:
import copy
import pandas as pd
import numpy as np
import re
from selenium import webdriver
import time
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import json
from langdetect import detect
from deep_translator import GoogleTranslator
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('words')
from nltk.corpus import words
import spacy
import requests
response = requests.get("https://translate.google.com/")
import gensim.corpora as corpora
nlp = spacy.load("en_core_web_sm")
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
session = requests.Session()
retry_strategy = Retry(total=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
response = session.get("https://translate.google.com")



# Tweets scraping

*Giving your own tweets(This might be a solution to the tweet limit in twitter).*

In [ ]:
ID = []
ID.append(str("@") + input("Your Twitter ID: "))
Password = input("Your Twitter Password: ")
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
tweets = []
times = []
UserTags = []
number = int(input("Number of Tweets you wish to apply with (50 < Suggsted): "))
for i in ID:
    browser = webdriver.Chrome()
    browser.get("https://twitter.com/i/flow/login")
    time.sleep(5)
    giris = browser.find_element(By.NAME,"text")
    giris.send_keys(ID)
    giris.send_keys(Keys.RETURN)
    time.sleep(1.2)
    browser.find_element(By.NAME,"password").send_keys(Password)
    browser.find_element(By.NAME,"password").send_keys(Keys.RETURN)
    time.sleep(3)
    browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input").send_keys(i)
    browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input").send_keys(Keys.RETURN)
    time.sleep(3)
    browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[3]/a").click()
    time.sleep(4)
    browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]/span/span[1]").click()
    articles = browser.find_elements(By.XPATH,"//article[@data-testid='tweet']")
    time.sleep(0.5)
    maxtweet = browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div/div/div/div/div/div[2]/div/div").text
    maxtweet = maxtweet[:-7]
    if maxtweet[-1] == "K":
        maxtweet = number + 100
    if len(str(maxtweet)) > 4:
        if maxtweet[-4] == ",":
            maxtweet = number +100
    else:
        maxtweet = int(maxtweet)
    tweets12 = []
    time.sleep(4)
    browser.find_element(By.TAG_NAME, value="body").send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    browser.find_element(By.TAG_NAME, value="body").send_keys(Keys.PAGE_DOWN)
    browser.find_element(By.TAG_NAME, value="body").send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    browser.find_element(By.TAG_NAME, value="body").send_keys(Keys.PAGE_DOWN)
    browser.find_element(By.TAG_NAME, value="body").send_keys(Keys.PAGE_DOWN)
    time.sleep(3)
    while True:
        for article in articles:
            UserTag = browser.find_element(By.XPATH,".//div[@data-testid='User-Name']").text
            UserTags.append(UserTag)
            
            Tweet = browser.find_element(By.XPATH,".//div[@data-testid='tweetText']").text
            tweets.append(Tweet)
            tweets12.append(Tweet)
            
            Timestamp = browser.find_element(By.XPATH,".//time").get_attribute("datetime")
            times.append(Timestamp)
        browser.find_element(By.TAG_NAME, value="body").send_keys(Keys.PAGE_DOWN)
        time.sleep(0.6)
        articles = browser.find_elements(By.XPATH,"//article[@data-testid='tweet']")
        tweets2 = list(set(tweets12))
        if len(tweets2) > number-1:
            break
        if maxtweet < number:
            if len(tweets2) > maxtweet/2 - 10:
                break
    browser.close()
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
for timee in range(0,len(times)):
    times[timee] = times[timee][0:10]
df = pd.DataFrame(zip(UserTags,times,tweets)
                  ,columns=['UserTags','TimeStamps','Tweets'])
df = df.drop_duplicates().reset_index(drop=True)
for dd in ID:
    df.iloc[number*ID.index(dd):(number*(ID.index(dd)+1)),0] = dd
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    for timee in range(0,len(times)):
        times[timee] = times[timee][0:10]
df = pd.DataFrame(zip(UserTags,times,tweets)
                  ,columns=['UserTags','TimeStamps','Tweets'])
df = df.drop_duplicates().reset_index(drop=True)
for dd in ID:
    df.iloc[number*ID.index(dd):(number*(ID.index(dd)+1)),0] = dd
with pd.ExcelWriter("project_tweet.xlsx") as writer:
    df.to_excel(writer, sheet_name="tweetler")
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
tweetler = pd.read_excel("project_tweet.xlsx")

# LDA Model

*Preprocessing and LDA Model*

In [ ]:
import requests
response = requests.get("https://translate.google.com/")
import gensim.corpora as corpora
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
session = requests.Session()
retry_strategy = Retry(total=5, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("https://", adapter)
response = session.get("https://translate.google.com")
########################################################
#Json function for LDA dict
file_path = 'LDA_dict.json'
def download_dictionary_to_pc(dictionary, file_path):
    json_string = json.dumps(dictionary)
    with open(file_path, 'w') as file:
        file.write(json_string)
with open(file_path, 'r') as file:
    LDA_dict = json.load(file)
tweetler = pd.read_excel("project_tweet.xlsx")
#Tag and Mention cleaning.
def remove_mentions_and_tags(text):
    text = re.sub(r"@\S*", "", text)
    text = re.sub(r"\n","",text)
    return re.sub(r"#\S*", "", text)
tweetler["Tweets"] = tweetler["Tweets"].apply(remove_mentions_and_tags)
print("Tags and mentions have been deleted...")
#Only alphabet.(TR dahil)
def keep_only_alphabet(text):
    return re.sub(r"[^a-zA-ZğüşıöçĞÜŞİÖÇı]", " ", text)
tweetler.Tweets = tweetler.Tweets.apply(keep_only_alphabet)
tweetler["Tweets"] = tweetler["Tweets"].apply(keep_only_alphabet)
print("Only the Alphabet...")
# #Language Detect
def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return "Unknown"
tweetler["Language"] = tweetler["Tweets"].apply(detect_language)
print("Languages are detected...")
# Delete languages other than tr or eng
tweetler = tweetler[tweetler['Language'].isin(['tr', 'en'])]
# Ingilizce çeviri(for TR)
tweetler = tweetler.reset_index(drop=True)
def translate_to_string(text, target_lang):
    translator = GoogleTranslator(source='auto', target=target_lang)
    translation = translator.translate(text)
    translated_text = str(translation)
    return translated_text
for i in range(0,len(tweetler)):
    if tweetler["Language"][i] == "tr":
        text = tweetler["Tweets"][i]
        target_lang = "en"  
        translated_text = translate_to_string(text, target_lang)
        tweetler["Tweets"][i] = translated_text
print("Translated to English...")
#Lowercase
for i in range(0,len(tweetler)):
    tweetler["Tweets"][i] = tweetler["Tweets"][i].lower()
print("Lowercased")
#Url cleaning
def remove_url(text):
  return re.sub(r"https?:\S*", "", text)
tweetler["Tweets"] = tweetler["Tweets"].apply(remove_url)
print("URL Cleaned...")
# Only English words contained.
def clean_non_english_words(text):
    english_words = set(words.words())
    cleaned_words = [word for word in text.split() if word.lower() in english_words]
    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text
for i in range(0,len(tweetler)):
    tweetler["Tweets"][i] = clean_non_english_words(tweetler["Tweets"][i])
# Lemmatization
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text
for i in range(0,len(tweetler)):
    text = tweetler["Tweets"][i]
    tweetler["Tweets"][i] = lemmatize_text(text)
#Delete Less Than 3 Character words
def remove_short_words(text):
    pattern = r'\b\w{1,2}\b'
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text
for i in range(0,len(tweetler)):
    tweetler["Tweets"][i] = remove_short_words(tweetler["Tweets"][i])
#Tokenize and clean stopwords
tweetler = tweetler[tweetler['Tweets'].str.strip().astype(bool)]
tweetler = tweetler.reset_index(drop=True)
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(data_words):
    nlp = spacy.load("en_core_web_sm")
    docs = [nlp(' '.join(map(str, words))) for words in data_words]
    filtered_words = [[token.text for token in doc if not token.is_stop] for doc in docs]
    return filtered_words
data = tweetler.Tweets.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
num_topics = lda_model.num_topics
num_words_per_topic = 5  # Number of words per topic

topics_matrix = lda_model.get_topics()
top_words = [[lda_model.id2word[word_id] for word_id, _ in lda_model.get_topic_terms(topic_id, topn=num_words_per_topic)]
             for topic_id in range(num_topics)]

# Example
for topic_id, words in enumerate(top_words):
    print(f"Topic {topic_id + 1}: {', '.join(words)}")
LDA_dict[ID[0]] = top_words
download_dictionary_to_pc(LDA_dict,file_path)

# LDA Calculating Scores

*Calculating similarity between your topics and the others.(Might lack further improvements)*

In [ ]:
import spacy

nlp = spacy.load('en_core_web_lg')

def n_similarity(word_set1, word_set2):
    tokens1 = nlp(" ".join(word_set1))
    tokens2 = nlp(" ".join(word_set2))
    return tokens1.similarity(tokens2)
scores = []
similarity_scores = 0
for i in range(0,10):
    word_set1 = LDA_dict[str(ID[0])][i]
    for k in LDA_dict:
        for j in range(0,10):
            word_set2 = LDA_dict[k][j]
            similarity_score = n_similarity(word_set1, word_set2)
            similarity_scores += similarity_score
        scores.append([f'{k}: ',similarity_scores/10])
        similarity_scores = 0
scores = pd.DataFrame(scores)
scores.columns = ["name","score"]
scores = pd.DataFrame(scores.groupby("name")["score"].mean())
scores = scores.sort_values(by="score",ascending=False)
scores = scores.reset_index()

# Pick a Friend

*Here the model matches you with people and asks you which profile do you want to observe.*

In [ ]:
for i in range(0,len(scores)):
    print(f'The {i+1}. Person is : {scores.iloc[i,0]} and his/her relevance_score is: {scores.iloc[i,1]}')
chosen = int(input("Whose tweets you wish to see? Please enter his/her number: "))
chosen1 = copy.copy(chosen)
chosen = scores.iloc[chosen-1,0]
chosen = chosen[:-2]
browser = webdriver.Chrome()
browser.get("https://twitter.com/i/flow/login")
time.sleep(5)
giris = browser.find_element(By.NAME,"text")
giris.send_keys(ID)
giris.send_keys(Keys.RETURN)
time.sleep(1.2)
browser.find_element(By.NAME,"password").send_keys(Password)
browser.find_element(By.NAME,"password").send_keys(Keys.RETURN)
time.sleep(3)
browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input").send_keys(chosen)
browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div[2]/div/div[2]/div/div/div/div[1]/div/div/div/form/div[1]/div/div/div/label/div[2]/div/input").send_keys(Keys.RETURN)
time.sleep(3)
browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div/div/div[1]/div[1]/div[2]/nav/div/div[2]/div/div[3]/a").click()
time.sleep(4)
browser.find_element(By.XPATH,"//*[@id='react-root']/div/div/div[2]/main/div/div/div/div/div/div[3]/section/div/div/div/div/div[1]/div/div/div/div/div[2]/div[1]/div[1]/div/div[1]/a/div/div[1]/span/span[1]").click()